In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets
from config import api_google
from config import api_owm
from pprint import pprint

In [2]:
df_path = f"../WeatherPy/output/city_desc2.csv"
df = pd.read_csv(df_path).drop(columns = "Unnamed: 0")[0:100]

In [3]:
df.head()

,City Name,Temperature (F),Humidity %,CLoudiness %,Wind Speed (mph)
0,bathsheba,79.23,82,16,20.49
1,melendugno,68.00,64,9,4.70
2,mecca,80.06,16,3,9.40
3,horadiz,69.80,52,0,6.93
4,inirida,74.57,96,95,1.16


In [4]:
humid = df["Humidity %"]

city = df["City Name"]

base_geo = "https://maps.googleapis.com/maps/api/geocode/json?address="
base_place = "https://maps.googleapis.com/maps/api/place/nearbysearch/"

In [5]:
lat_lon=[]

In [6]:
for name in city:
    a=requests.get(base_geo+name + "&key="+api_google).json()
    
    lat_lon.append(a["results"])

In [7]:
pprint(lat_lon[10][0]["address_components"])

[{'long_name': 'Impruneta',
  'short_name': 'Impruneta',
  'types': ['locality', 'political']},
 {'long_name': 'Impruneta',
  'short_name': 'Impruneta',
  'types': ['administrative_area_level_3', 'political']},
 {'long_name': 'Metropolitan City of Florence',
  'short_name': 'FI',
  'types': ['administrative_area_level_2', 'political']},
 {'long_name': 'Tuscany',
  'short_name': 'Tuscany',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'Italy', 'short_name': 'IT', 'types': ['country', 'political']},
 {'long_name': '50023', 'short_name': '50023', 'types': ['postal_code']}]


In [8]:
lat=[]
lon=[]
country=[]
j=0

In [9]:
for i in range(0, len(lat_lon)):
    lat.append (lat_lon[i][0]["geometry"]["location"]["lat"])
    lon.append (lat_lon[i][0]["geometry"]["location"]["lng"])


In [10]:
country

[]

In [11]:
df["lat"]=lat
df["lon"]=lon

In [12]:
#configure gmaps
gmaps.configure(api_key=api_google)

In [13]:
locations = df[['lat','lon']]
weights = df['Humidity %']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights) )
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
#now, let's find ideal vacation spot!
df2 = df[(df["Temperature (F)"]>= 70) & (df["Temperature (F)"]<= 80) & (df["Wind Speed (mph)"]< 10) & (df["CLoudiness %"]< 5) ].reset_index(drop=True)

In [15]:
df2

,City Name,Temperature (F),Humidity %,CLoudiness %,Wind Speed (mph),lat,lon
0,inderborskiy,74.23,32,0,9.98,48.548962,51.740292
1,tabuk,73.11,28,0,8.19,28.383508,36.566191
2,brindisi,72.21,75,0,7.14,40.632728,17.941762
3,inderborskiy,74.23,32,0,9.98,48.548962,51.740292
4,tabuk,73.11,28,0,8.19,28.383508,36.566191
5,brindisi,72.21,75,0,7.14,40.632728,17.941762


In [16]:
hotel=[]
hotel_name=[]
hotel_lat=[]
hotel_lon=[]
hotel_country=[]
hotel_desc=[]



In [17]:
cor=[]

for i in range(0,len(df2)):
    a=str(df2.iloc[i]["lat"])+","+str(df2.iloc[i]["lon"])
    cor.append(a)

df2["coord"]=cor


In [18]:
# This will return the country name instead of country code, which I think is better!

url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in range(0,len(df2)):
    params={"key":api_google,
            "location":df2.loc[i]["coord"],
            "radius":5000,
           "keyword":"hotel"}
    a = requests.get(url, params=params).json()
    hotel_name.append(a["results"][0]["name"])
    b = a["results"][0]["geometry"]["location"]["lat"]
    c = a["results"][0]["geometry"]["location"]["lng"]
    d = a["results"][0]["plus_code"]["compound_code"].split()[-1]
    hotel_lat.append(b)
    hotel_lon.append(c)
    hotel_country.append(d)

In [19]:
#df2["Hotel Location"]=hotel_loc

In [20]:
len(hotel_name)

6

In [21]:
df2["Hotel Name"]=hotel_name



In [22]:
#get hotel data

#    params={"key":api_google,
#            "location":df2.loc[0]["coord"],
#            "radius":5000,
#           "keyword":"hotel"}
#    a = requests.get(url, params=params).json()


In [23]:
#preparing for content
marker_content=[]

for i in range(0, len(df2)):
    a={"Hotel Name":hotel_name[i],
       "City":df2["City Name"][i],
       "Country":hotel_country[i]}
    marker_content.append(a)
    
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in marker_content]


In [24]:
locations = df[['lat','lon']]
weights = df['Humidity %']
marker_locations=df2[['lat','lon']]


fig = gmaps.figure()
markers=gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights) )
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))